In [1]:
import copy
import time
current_time = time.localtime()
# 格式化为指定格式的字符串
formatted_time = time.strftime("%m%d%H%M", current_time)
from logger_config import setup_logger
logger = setup_logger(formatted_time)

# 读取plot_chain.json中的内容到字典
import json
with open("raw_plot_chain.json", "r", encoding='utf-8') as f:
    original_outline = json.load(f)

plot_chain = original_outline["plot_chain"]

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model='ep-',
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key="",
    base_url="",
)

model_pools =[
    "ep-",
    "ep-",
    "ep-",
    "ep-",
    "ep-",
]

r1_api = ""

In [3]:
to_do_plot_chain = plot_chain[7:]
num = int(round(len(to_do_plot_chain)/3))

In [4]:
original_outline

{'characters': {'三月七': {'外观': '粉色短发的少女开拓者',
   '简介': '一名精灵古怪的少女，因为羡慕武侠故事中行侠仗义的大侠，所以选择向彦卿和云璃拜师学剑。',
   '性格': '天真烂漫，好奇心强，对剑术充满热情。',
   '经历': []},
  '彦卿': {'外观': '浅黄发色的少年剑士',
   '简介': '意气飞扬的云骑骁卫，仙舟「罗浮」最强剑士。为剑生亦为剑痴，当彦卿手中握剑时，无人敢小看这位尚在总角之年的天才。或许能让手中宝剑微微收敛锋芒的，只有时间。',
   '性格': '锋芒毕露的自信中裹挟少年躁动，胜负欲炽烈如剑芒，行事带三分莽撞却暗藏机敏。骨子里浸着桀骜不驯的锐气，对强者既存挑战执念又萌生敬慕，赤诚里混着偏执。兼具不谙世事的纯粹与早慧通透的悟性，似未开刃的利剑，既有刺破天穹的锋芒，亦存割伤自身的脆弱。',
   '经历': ['我自幼由景元将军收养，打记事起，便追随将军左右。未曾见过亲生父母，也不清楚自己的身世。希望能为罗浮保驾护航、为景元分忧克难，以回应将军的期许，报答他的教养之恩。',
    '景元在罗浮召开演武仪典，我受邀请前来参加。刚到不久便遇见了开拓者一行人，见面时开拓者似乎因为担心碰面的人有危险，捏了我的脸。随后码头上突然出现一群步离狼卒引发暴乱，我出手击倒大部分步离人，却因一名少女的撞击导致飞剑失准，未能击杀最后一个逃走的步离人。那少女用我的飞剑钉住步离人后以重剑将其击败，却直接拿走了我的飞剑离去。',
    '我与开拓者一同去见景元和朱明的烛渊将军怀炎，却发现先前那名少女竟是怀炎的孙女云璃。得知此事后，我要求云璃归还飞剑，但她拒绝，称我配不上这把飞剑，还说飞剑对她诉说了对我的不满。我们因此产生不和。',
    '我们排查被步离人袭击的星际和平公司商队，发现他们正在运输博识学会开发的实验兵器咆哮灵柩，同时认识了新上任的丹鼎司司鼎灵砂。随灵砂前往丹鼎司时再次遇到云璃，我们又一次发生争执。在灵砂的建议下，我们进行了一场狩猎孽物的比赛，但比赛后仍互不服气，最终再次开战。曜青天击将军飞霄空手接住了我们的重剑和飞剑，制止了争斗并调解。云璃虽归还了飞剑，但仍态度冷淡，声称会在演武仪典上战胜我。',
    '怀炎安排我和云璃在演武仪典前共同培训一位弟子，并至少让其击败一个挑战者。三月七被选

In [5]:
from agent import ScriptWriter
sw = ScriptWriter(original_outline, r1_api, model_pools, 7, 3)

D:\Tools\conda\envs\api\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
complete_outline = sw.create_branches(llm, model_pools)

100%|██████████| 1/1 [03:01<00:00, 181.90s/it]


In [13]:
# 将plot—chain信息保存到本地
import copy
plot_chain_dict = {}
outline_info = copy.deepcopy(sw.outline)
for plot_chain in sw.outline['plot_chains']:
    plot_chain_dict[plot_chain.info] = {
        'all_plot': plot_chain.all_plot,
        'generated_index': plot_chain.generated_index,
        'todo_fork_point': plot_chain.todo_fork_point,
        'fork_stream_cache': plot_chain.fork_stream_cache,
        'children': [children.info for children in plot_chain.children] if plot_chain.children else []
    }
    
outline_info['plot_chains'] = plot_chain_dict
with open(f"{sw.current_time}/outline.json", "w", encoding='utf-8') as f:
    json.dump(outline_info, f, ensure_ascii=False, indent=4)
